### 1.목표
- 네이버 영화리뷰 데이터 셋을 이용해서 긍정/부정 분류기를 만들어 보자!
- TF-IDF 방법을 적용해서 토큰화 작업을 해보자
- konlpy 한국어 형태소 분석기를 설치하고 활용해보자
- 단어별 긍/부정 정보를 시각화해서 확인해보자
  

In [3]:
# 1. java 버전 확인하기
# cmd 창에서 java -version 확인하기 -> 1.7버전 이상인지 확인해보자
# 만약 java가 없는 경우 설치 후 시스템 환경변수 편집까지 해주자

In [4]:
# # 2.Jpype 설치
# # konlpy는 자바 기반의 라이브러리 -> 파이썬 환경에서 원활하게 사용하도록 해주는 별도의 도구
# !pip install jpype1

In [5]:
# # 3.konlpy 설치
# # konlpy : 한국어 형태서 분석기를 모아둔 라이브러리이다.
# !pip install konlpy

In [10]:
from konlpy.tag import Kkma
import pandas as pd

In [7]:
kkma = Kkma()

### 2.네이버 데이터 로딩

In [19]:
# 훈련 데이터 셋 - ratings_train.csv
# 평가용 데이터 셋 - ratings_test.csv

df_train= pd.read_csv("./data/ratings_train.csv")
df_test= pd.read_csv("./data/ratings_test.csv")

In [24]:
# 데이터 확인
print("훈련 데이터 셋 크기 :",df_train.shape)
print("평가용 데이터 셋 크기 :",df_test.shape)

훈련 데이터 셋 크기 : (150000, 3)
평가용 데이터 셋 크기 : (50000, 3)


In [21]:
df_train.head()


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [22]:
df_test.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [26]:
# 결측치 확인
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [42]:
df_train['document'].head()

0                                  아 더빙.. 진짜 짜증나네요 목소리
1                    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                        교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
Name: document, dtype: object

In [43]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


- train  데이터와 test 데이터에 document 컬럼에 각각 5개/3개의 결측치가 확인되었다.
- 결측치의 형태를 보니 직접적인 리뷰 데이터 -> 별도로 채워줄 수 있는 방법이 없다.

In [46]:
# 결측치가 있는 데이터를 삭제하는 함수
# dropna(axis=0, 1, inplace=)

df_train.dropna(axis=0, inplace=True)
df_test.dropna(axis=0, inplace=True)

In [51]:
# 결측치 제거 확인 - train
df_train.info()
df_train.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


id          0
document    0
label       0
dtype: int64

In [52]:
# 결측치 제거 확인 - test
df_test.info()
df_test.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


id          0
document    0
label       0
dtype: int64

#### 3.문제/정답으로 분할


In [56]:
df_train.head()

# id : 리뷰를 작성한 사용자의 정보
# document : 실제 리뷰 내용
# label : 긍정/부정을 표시하는 정답 레이블 -> 긍정 : 1, 부정 : 0

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [57]:
text_train = df_train['document']
y_train = df_train['label']

text_test = df_test['document']
y_test = df_test['label']

In [58]:
print(text_train.shape)
print(y_train.shape)
print(text_test.shape)
print(y_test.shape)


(149995,)
(149995,)
(49997,)
(49997,)


'紐⑦넗濡쒕씪異⑹쟾濡'